In [ ]:
# Import Python packages used in this notebook
import streamlit as st
import altair as alt

# Pre-installed libraries that comes with the notebook
import pandas as pd
import numpy as np

# Package that we just added
import matplotlib.pyplot as plt

If you are interested in the data I used, you can find it in Kaggle: 
* https://www.kaggle.com/datasets/nilimajauhari/glassdoor-analyze-gender-pay-gap

In [ ]:
/*CREATE stage RAW_DATA;
--add file to stage from data tab https://www.kaggle.com/datasets/nilimajauhari/glassdoor-analyze-gender-pay-gap
LIST RAW_DATA;
CREATE TABLE SALARY_DATA (
JobTitle VARCHAR(255),
Gender VARCHAR(20),
Age NUMBER,
PerfEval NUMBER,
Education VARCHAR(255),
Dept VARCHAR(255),
Seniority NUMBER,
BasePay NUMBER,
Bonus NUMBER
);
COPY INTO SALARY_DATA
FROM @BUILD_RAW_DATA
FILE_FORMAT = (TYPE = CSV SKIP_HEADER = 1);*/

In [ ]:
SELECT * 
FROM SALARY_DATA

In [ ]:
df = Select_star.to_pandas()
df["TOTAL_SALARY"]=df["BASEPAY"]+df["BONUS"]

In [ ]:
df.describe()

In [ ]:
fig, ax = plt.subplots(figsize = (6,3))
plt.tick_params(left = False, right = False , labelleft = False) 

min_salary = min(df['TOTAL_SALARY'])
max_salary = max(df['TOTAL_SALARY'])
bin_size = 10000

price = df["TOTAL_SALARY"]
price.plot(kind = "hist", density = True, bins = range(min_salary, max_salary + bin_size, bin_size))
price.plot(kind="kde", color='#c44e52')


# Calculate percentiles
median = price.median()
ax.axvline(median,0, color='#dd8452', ls='--')
ax.text(median,0.8, f'Median: {median:.2f}  ',
        ha='right', va='center', color='#dd8452', transform=ax.get_xaxis_transform())

# Make our chart pretty
plt.style.use("bmh")
plt.title("Total Salary Distribution")
plt.xlabel("Total Salary (binned)")
left, right = plt.xlim()   
plt.xlim((0, right))  
# Remove ticks and spines
ax.tick_params(left = False, bottom = False)
for ax, spine in ax.spines.items():
    spine.set_visible(False)

plt.show()

In [ ]:
# Parameters
#bin_size = 10000

# Split the DataFrame by GENDER
male_salaries = df[df['GENDER'] == 'Male']['TOTAL_SALARY']
female_salaries = df[df['GENDER'] == 'Female']['TOTAL_SALARY']

# Determine the range of bins
#min_salary = min(df['BASEPAY'])
#max_salary = max(df['BASEPAY'])
bins = range(min_salary, max_salary + bin_size, bin_size)

# Plot histograms
plt.figure(figsize=(6, 4))

plt.hist(male_salaries, bins=bins, alpha=0.5, label='Male', color='green', edgecolor='black')
plt.hist(female_salaries, bins=bins, alpha=0.5, label='Female', color='blue', edgecolor='black')

# Add labels and legend
plt.title('Salary Distribution by Gender')
plt.xlabel('Salary')
plt.ylabel('Frequency')
plt.legend(bbox_to_anchor=(1, 1))
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Show the plot
plt.tight_layout()
plt.show()

I wonder - how does it look on different levels on seniority?

In [ ]:
seniority = 3 

In [ ]:
SELECT * FROM SALARY_DATA where SENIORITY > {{seniority}}

## Creating an interactive app with Streamlit 🪄

Putting this all together, let's build a Streamlit app to explore how different parameters impacts the shape of the data distribution histogram.

In [ ]:
import streamlit as st
st.markdown("# Move the slider to adjust and watch the results update! 👇")
col1, col2 = st.columns(2)
with col1:
    seniority = st.slider('SENIORITY threshold',1,5,1) 

# Read table from Snowpark and plot the results
from snowflake.snowpark.context import get_active_session
session = get_active_session()
df = session.sql(
    f"""
    SELECT * FROM SALARY_DATA where SENIORITY = {seniority};
    """
    ).to_pandas()

# Split the DataFrame by GENDER
male_salaries = df[df['GENDER'] == 'Male']['BASEPAY']
female_salaries = df[df['GENDER'] == 'Female']['BASEPAY']

# Determine the range of bins
min_salary = min(df['BASEPAY'])
max_salary = max(df['BASEPAY'])
bins = range(min_salary, max_salary + bin_size, bin_size)

# Plot histograms
plt.figure(figsize=(6, 4))

plt.hist(male_salaries, bins=bins, alpha=0.5, label='Male', color='green', edgecolor='black')
plt.hist(female_salaries, bins=bins, alpha=0.5, label='Female', color='blue', edgecolor='black')

# Add labels and legend
plt.title('Salary Distribution by Gender')
plt.xlabel('Salary')
plt.ylabel('Frequency')
plt.legend(bbox_to_anchor=(1, 1))
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Show the plot
plt.tight_layout()
plt.show()

df.describe()

In [ ]:
with agg as (
SELECT JOBTITLE, GENDER, AVG(BASEPAY + BONUS) as TOTAL_SALARY
FROM SALARY_DATA
group by 1,2
order by 1
),
pvt as (
select * from agg PIVOT (sum (TOTAL_SALARY) for GENDER IN (ANY ORDER BY GENDER)) as G (JOB, F, M)
)
select *, M-F as gap from pvt where F < M order by 4 desc

I would like to understand if the overall salaries difference is statistically significant. 
* H0 -  group 'Female' earns on average same as group 'Male'.
* H1 - group 'Female' earns on average less than group 'Male'.
* Given the salaries data is normally distributed, I shall proceed with one-sided (less than) t-test. I start with finding means and group sizes for each gender:

In [ ]:
SELECT GENDER, AVG(BASEPAY + BONUS) as TOTAL_SALARY, count(*) as sample_size
FROM SALARY_DATA
group by 1

I proceed with power analysis: given I want to detect even small difference (d = 0.2), with statistical significance 95% (alpha = 0.05) and power 80%, what sample size would I need?

In [ ]:
from statsmodels.stats.power import TTestPower
power = TTestPower()
sample_size = power.solve_power(effect_size=0.2, alpha=0.05, power=0.8, alternative='larger')
print(sample_size)

Output solved for sample size = 155.9256735887691, while my groups are ±500 each, I am satisfied with the result.

I am now collecting the necessary statistics for the analysis: pooled STD and overall SE.

In [ ]:
WITH stats AS (
    SELECT
        GENDER,
        COUNT(1) AS n,                     -- Group size
        AVG(BASEPAY + BONUS) AS mean_salary,             -- Mean salary
        VAR_POP(BASEPAY + BONUS) AS variance_salary,      -- Variance of salary
        STDDEV_POP(BASEPAY + BONUS) AS stddev_salary,
        STDDEV_POP(BASEPAY + BONUS)/SQRT(COUNT(1)) AS SE
    FROM
        SALARY_DATA
    GROUP BY
        GENDER
),
pooled_stats AS (
    SELECT
        SUM((n - 1) * variance_salary) AS numerator,    -- Weighted sum of variances
        SUM(n) - 2 AS denominator,                      -- Degrees of freedom
        SUM(n) AS total_n
    FROM
        stats
)
SELECT
    SQRT(numerator / denominator) AS pooled_std_dev,    -- Final pooled standard deviation
    numerator / denominator AS pooled_var,
    SQRT((numerator / denominator)/total_n) AS overall_std_error
FROM
    pooled_stats;

Given the above and as my data follows normal distribution, I perform t-test for determining significance.

In [ ]:
SELECT (96416 - 104918)/783

As the outcome t-value is -10. For one-sided left side test, after consulting the t-table, p-value is close to 0. This means that the result is strongly significant - the H1 should be accepted (group 'Female' earns on average less than group 'Male').

Lastly, once we understood that the result is significant, can we interpret the difference: is it "large" or "small"? I am going to use Cohen d for that:

In [ ]:
SELECT (104918 - 96416)/24783

The value 0.34 is between 'medium' and 'large' according to standart interpretation. As a next step, correlation and linear regression could be used to explore this futher.